In [6]:
import pyomo as pyo
import pandas as pd
import numpy as np
from data_processing import load_all_data
from calcula_fpo import calcular_fpo

In [2]:
data = load_all_data()
D_GEN = data['D_GEN']
D_LIN = data['D_LIN']
D_LOAD = data['D_LOAD']

In [5]:
#Deixando os dados em horas e nos formatos conhecidos
#Dados de Geração
D_GEN['Lambda'] = D_GEN['Falhas/ano'] / 8760 # 8760 horas em um ano
D_GEN['MTTF'] = 1/D_GEN['Lambda'] #Tempo médio entre falhas
D_GEN['mu'] = 1/D_GEN['Tempo de reparo(h)'] #Taxa de reparo/hora
D_GEN['A'] = D_GEN['mu']/(D_GEN['mu'] + D_GEN['Lambda']) #Disponibilidade
D_GEN['U'] = 1 - D_GEN['A'] #Indisponibilidade

#Dados de Linha
D_LIN['Lambda'] = D_LIN['Falhas/(ano.milha)'] * D_LIN['Comprimento'] / 8760 # 8760 horas em um ano
D_LIN['MTTF'] = 1 / D_LIN['Lambda'] #Tempo médio entre falhas
D_LIN['mu'] = 1 / D_LIN['Tempo de Reparo'] #Taxa de reparo/hora
D_LIN['A'] = D_LIN['mu'] / (D_LIN['mu'] + D_LIN['Lambda']) #Disponibilidade
D_LIN['U'] = 1 - D_LIN['A'] #Indisponibilidade

#Numero de componentes
Ng = len(D_GEN)
Nl = len(D_LIN)
Nc = len(D_LOAD)

# Preparando listas de probabilidades
prob_geradores = D_GEN[["A", "U"]].values  # Matriz de [p(A), p(U)] para cada gerador
prob_linhas = D_LIN[["A", "U"]].values    # Matriz de [p(A), p(U)] para cada linha

In [10]:
def calcular_estatisticas(resultados):
    if len(resultados) > 1:
        # Calcular a média
        media = sum(resultados) / len(resultados)
        
        # Calcular a variância (usando a fórmula da variância amostral)
        variancia = sum((x - media) ** 2 for x in resultados) / (len(resultados) - 1)  # ddof=1 para variância amostral
        
        # Calcular a variância dividida pelo tamanho da lista
        variancia_media = variancia / len(resultados)
        
        # Calcular o Beta
        beta = (variancia_media ** 0.5) / media if media != 0 else 10
        return media, variancia, beta
    else:
        return 10, 10, 10

In [12]:
# Configurações iniciais
N_MAX = 100  # Número máximo de amostras
tol = 0.1  # Critério de tolerância para convergência
beta = 10  # Inicialização do erro relativo
N = 0  # Contador de amostras

# Vetores para armazenar resultados
resultados_LOLP = []
resultados_LOLE = []
resultados_EPNS = []
resultados_EENS = []
resultados_LOLF = []
resultados_LOLD = []

# Loop principal da SMC Não Sequencial
while beta > tol and N < N_MAX:
    N += 1  # Incrementa o número de amostras
    
    # Passo 1: Sortear os estados dos componentes
    estados_geradores = [np.random.choice([1, 0], p=p) for p in prob_geradores]
    estados_linhas = [np.random.choice([1, 0], p=p) for p in prob_linhas]
    
    # Passo 2: Calcular o corte de carga
    corte_carga = calcular_fpo(estados_geradores, estados_linhas, False)
    
    # Atualizar estatísticas do sistema
    if corte_carga > 0:
        energia_nao_suprida = corte_carga
        horas_com_corte = 1  # Cada amostra representa 1 hora (estado estático)
        eventos_corte = 1
    else:
        energia_nao_suprida = 0
        horas_com_corte = 0
        eventos_corte = 0
    
    # Armazenar resultados
    resultados_LOLP.append(horas_com_corte)
    resultados_LOLE.append(horas_com_corte)
    resultados_EPNS.append(corte_carga)
    resultados_EENS.append(energia_nao_suprida)
    resultados_LOLF.append(eventos_corte)
    if eventos_corte > 0:
        resultados_LOLD.append(horas_com_corte / eventos_corte)

    # Passo 3: Verificar convergência
    if N > 2:
        # Calcular médias e beta para convergência
        media_LOLP, var_LOLP, beta_LOLP = calcular_estatisticas(resultados_LOLP)
        media_LOLE, var_LOLE, beta_LOLE = calcular_estatisticas(resultados_LOLE)
        media_EPNS, var_EPNS, beta_EPNS = calcular_estatisticas(resultados_EPNS)
        media_EENS, var_EENS, beta_EENS = calcular_estatisticas(resultados_EENS)
        media_LOLF, var_LOLF, beta_LOLF = calcular_estatisticas(resultados_LOLF)
        media_LOLD, var_LOLD, beta_LOLD = calcular_estatisticas(resultados_LOLD)

        # Determinar o maior beta (critério de convergência)
        beta = max([beta_LOLP, beta_LOLE, beta_EPNS, beta_EENS, beta_LOLF])

# Calcular desvio padrão e erro padrão
std_LOLP = np.std(resultados_LOLP)
std_LOLE = np.std(resultados_LOLE)
std_EPNS = np.std(resultados_EPNS)
std_EENS = np.std(resultados_EENS)
std_LOLF = np.std(resultados_LOLF)
std_LOLD = np.std(resultados_LOLD)

erro_padrao_LOLP = std_LOLP / np.sqrt(N)
erro_padrao_LOLE = std_LOLE / np.sqrt(N)
erro_padrao_EPNS = std_EPNS / np.sqrt(N)
erro_padrao_EENS = std_EENS / np.sqrt(N)
erro_padrao_LOLF = std_LOLF / np.sqrt(N)
erro_padrao_LOLD = std_LOLD / np.sqrt(N)

# Resultados finais
print(f"Expected Energy Not Supplied (EENS): ({media_EENS/1000:.2f} ± {erro_padrao_EENS/1000:.3f}) GWh/ano")
print(f"Expected Power Not Supplied (EPNS): ({media_EPNS:.2f} ± {erro_padrao_EPNS:.3f}) MW")
print(f"Loss of Load Duration (LOLD): ({media_LOLD:.2f} ± {erro_padrao_LOLD:.3f}) horas/eventos de corte")
print(f"Loss of Load Expectation (LOLE): ({media_LOLE:.2f} ± {erro_padrao_LOLE:.3f}) horas/ano")
print(f"Loss of Load Probability (LOLP): ({media_LOLP*100:.2f} ± {erro_padrao_LOLP*100:.3f}) %")
print(f"Loss of Load Frequency (LOLF): ({media_LOLF:.2f} ± {erro_padrao_LOLF:.3f}) eventos de corte/ano")

Expected Energy Not Supplied (EENS): (0.01 ± 0.004) GWh/ano
Expected Power Not Supplied (EPNS): (7.67 ± 3.540) MW
Loss of Load Duration (LOLD): (1.00 ± 0.000) horas/eventos de corte
Loss of Load Expectation (LOLE): (0.09 ± 0.029) horas/ano
Loss of Load Probability (LOLP): (9.00 ± 2.862) %
Loss of Load Frequency (LOLF): (0.09 ± 0.029) eventos de corte/ano
